In [2]:
# !pip install rosbags

from pathlib import Path
from itertools import islice
from rosbags.highlevel import AnyReader

BAG_PATH = Path(r"D:\FYP\Datasets\github multisensor\Mavic2.bag")
AUDIO_TOPICS = ["/audio1/audio", "/audio2/audio", "/audio3/audio", "/audio4/audio"]
MAX_MSGS_PER_TOPIC = 300  # peek only the first N messages

def bytes_len(x):
    if x is None: return 0
    if isinstance(x, (bytes, bytearray)): return len(x)
    if isinstance(x, (list, tuple)) and (not x or isinstance(x[0], int)): return len(x)
    return 0

def is_mp3_head(b: bytes) -> bool:
    return b.startswith(b"ID3") or (len(b) >= 2 and b[0] == 0xFF and (b[1] & 0xE0) == 0xE0)

with AnyReader([BAG_PATH]) as reader:
    conns = {c.topic: c for c in reader.connections if c.topic in AUDIO_TOPICS}
    if not conns:
        print("None of the AUDIO_TOPICS are present in this bag.")
    else:
        print("Checking topics (capped at", MAX_MSGS_PER_TOPIC, "msgs/topic):")
        for topic in AUDIO_TOPICS:
            conn = conns.get(topic)
            if conn is None:
                print(f" - {topic:<15} -> not in bag")
                continue

            has_data, checked, head = False, 0, b""
            for _c, ts, raw in islice(reader.messages(connections=[conn]), MAX_MSGS_PER_TOPIC):
                checked += 1
                msg = reader.deserialize(raw, conn.msgtype)
                d = getattr(msg, "data", None)
                if bytes_len(d) > 0:
                    b = d if isinstance(d, (bytes, bytearray)) else bytes(d)
                    head = b[:16]
                    has_data = True
                    break

            tag = "HAS AUDIO" if has_data else f"empty (checked {checked})"
            hint = " (mp3-like)" if has_data and is_mp3_head(head) else ""
            print(f" - {topic:<15} -> {tag}{hint}")


Checking topics (capped at 300 msgs/topic):
 - /audio1/audio   -> empty (checked 300)
 - /audio2/audio   -> empty (checked 300)
 - /audio3/audio   -> empty (checked 300)
 - /audio4/audio   -> empty (checked 300)
